<a href="https://colab.research.google.com/github/Mostafa-Hamidifard/cvx-optimization/blob/main/HW5_p2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

please ensure that you added pwl_fit_data.csv to your colab 

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('/content/pwl_fit_data.csv')
x = df['x'].values
y = df['x'].values
x = x.reshape((-1,1))
y = y.reshape((-1,1))

In [ ]:
import cvxpy as cvx
internal_knot_number = 3
a_ = np.linspace(0,1,internal_knot_number+2,True).reshape((-1,1))
K = internal_knot_number + 1
alpha_ = cvx.Variable((K,1))
beta_ = cvx.Variable((K,1))
const1 = []
const2 = []
for i in range(K-1):
    const1+= [alpha_[i] <= alpha_[i+1]]
    const2+= [alpha_[i][0] * a_[i+1][0] + beta_[i][0] == alpha_[i+1][0] * a_[i+1][0] + beta_[i+1][0]]
constraints = const1 + const2

obj = 0
for i in range(len(y)):
    le = alpha_[0][0]*x[i]+beta_[0][0]
    for j in range(1,K-1):
        le =  cvx.maximum(le,alpha_[j][0]*x[i]+beta_[j][0])
    obj += cvx.norm2(le-y[i])

objective = cvx.Minimize(obj)
problem = cvx.Problem(objective,constraints)
problem.solve()

Expression(CONVEX, UNKNOWN, ())